# תיקון שגיאות - קודים לינאריים והמינג

מחברת אינטראקטיבית להבנת קודים לזיהוי ותיקון שגיאות.

## 1. מושגי יסוד

- **k** = מספר ביטים של מידע
- **n** = אורך קוד (מידע + בדיקה)
- **r** = ביטי בדיקה (n - k)
- **מרחק המינג** = מספר הביטים השונים בין שתי מילים

In [ ]:
def hamming_distance(s1, s2):
    """מרחק המינג בין שתי מחרוזות"""
    if len(s1) != len(s2):
        raise ValueError("אורכים שונים")
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

# דוגמאות
print(f"מרחק בין '1010' ו-'1001': {hamming_distance('1010', '1001')}")
print(f"מרחק בין '0000' ו-'1111': {hamming_distance('0000', '1111')}")
print(f"מרחק בין '1100' ו-'1100': {hamming_distance('1100', '1100')}")

## 2. מרחק מינימלי של קוד

In [ ]:
def min_distance(codewords):
    """מרחק מינימלי בין כל מילות הקוד"""
    min_dist = float('inf')
    for i in range(len(codewords)):
        for j in range(i + 1, len(codewords)):
            d = hamming_distance(codewords[i], codewords[j])
            min_dist = min(min_dist, d)
    return min_dist

# קוד לדוגמה
code = ['0000', '0111', '1011', '1100']
print(f"מילות קוד: {code}")
print(f"מרחק מינימלי: {min_distance(code)}")

# יכולות הקוד
d = min_distance(code)
detect = d - 1
correct = (d - 1) // 2
print(f"\nיכולות:")
print(f"  זיהוי עד {detect} שגיאות")
print(f"  תיקון עד {correct} שגיאות")

## 3. סיבית זוגיות (Parity Bit)

In [ ]:
def add_parity(data):
    """הוספת סיבית זוגיות"""
    count = data.count('1')
    parity = '1' if count % 2 == 1 else '0'
    return data + parity

def check_parity(code):
    """בדיקת זוגיות"""
    return code.count('1') % 2 == 0

# דוגמאות
data = "1010"
encoded = add_parity(data)
print(f"מידע: {data}")
print(f"עם זוגיות: {encoded}")
print(f"בדיקה: {'תקין' if check_parity(encoded) else 'שגיאה'}")

# הכנסת שגיאה
corrupted = encoded[:2] + ('0' if encoded[2] == '1' else '1') + encoded[3:]
print(f"\nעם שגיאה: {corrupted}")
print(f"בדיקה: {'תקין' if check_parity(corrupted) else 'שגיאה!'}")

## 4. קוד המינג (7,4)

- 4 ביטי מידע (d1, d2, d3, d4)
- 3 ביטי בדיקה (p1, p2, p3)
- מתקן שגיאה בודדת

In [ ]:
def hamming_encode(data):
    """
    קידוד המינג (7,4)
    סדר: p1 p2 d1 p3 d2 d3 d4
    מיקומים: 1  2  3  4  5  6  7
    """
    if len(data) != 4:
        raise ValueError("נדרשים 4 ביטים")
    
    d1, d2, d3, d4 = [int(b) for b in data]
    
    # חישוב ביטי בדיקה
    p1 = d1 ^ d2 ^ d4  # מיקומים 1,3,5,7
    p2 = d1 ^ d3 ^ d4  # מיקומים 2,3,6,7
    p3 = d2 ^ d3 ^ d4  # מיקומים 4,5,6,7
    
    # הרכבת הקוד: p1 p2 d1 p3 d2 d3 d4
    return f"{p1}{p2}{d1}{p3}{d2}{d3}{d4}"

# דוגמאות
test_data = ['0000', '1010', '1111', '0101']
print("קידוד המינג (7,4):")
print("מידע -> קוד")
for data in test_data:
    encoded = hamming_encode(data)
    print(f"{data}  -> {encoded}")

## 5. פענוח ותיקון שגיאות

In [ ]:
def hamming_decode(code):
    """
    פענוח ותיקון המינג (7,4)
    """
    if len(code) != 7:
        raise ValueError("נדרשים 7 ביטים")
    
    bits = [int(b) for b in code]
    p1, p2, d1, p3, d2, d3, d4 = bits
    
    # חישוב סינדרום (syndrome)
    s1 = p1 ^ d1 ^ d2 ^ d4
    s2 = p2 ^ d1 ^ d3 ^ d4
    s3 = p3 ^ d2 ^ d3 ^ d4
    
    # מיקום השגיאה (0 = אין שגיאה)
    error_pos = s1 * 1 + s2 * 2 + s3 * 4
    
    print(f"קוד שהתקבל: {code}")
    print(f"סינדרום: ({s1}, {s2}, {s3}) = מיקום {error_pos}")
    
    # תיקון אם יש שגיאה
    if error_pos > 0:
        print(f"נמצאה שגיאה במיקום {error_pos}!")
        bits[error_pos - 1] ^= 1  # תיקון (flip)
        print(f"קוד מתוקן: {''.join(str(b) for b in bits)}")
    else:
        print("לא נמצאו שגיאות")
    
    # חילוץ המידע
    data = f"{bits[2]}{bits[4]}{bits[5]}{bits[6]}"
    return data

# דוגמה ללא שגיאה
print("=== ללא שגיאה ===")
data = hamming_decode("1011010")
print(f"מידע: {data}\n")

# דוגמה עם שגיאה
print("=== עם שגיאה ===")
correct_code = "1011010"
# שגיאה במיקום 5
corrupted = correct_code[:4] + ('0' if correct_code[4] == '1' else '1') + correct_code[5:]
data = hamming_decode(corrupted)
print(f"מידע: {data}")

## 6. המינג מורחב (8,4) - SECDED

In [ ]:
def hamming_84_encode(data):
    """קידוד המינג (8,4) - Single Error Correct, Double Error Detect"""
    code_7 = hamming_encode(data)
    # הוספת ביט זוגיות כללי
    overall_parity = str(code_7.count('1') % 2)
    return code_7 + overall_parity

def hamming_84_decode(code):
    """פענוח המינג (8,4)"""
    if len(code) != 8:
        raise ValueError("נדרשים 8 ביטים")
    
    # בדיקת זוגיות כללית
    overall_parity = code.count('1') % 2
    
    # חישוב סינדרום מ-7 הביטים הראשונים
    bits = [int(b) for b in code[:7]]
    p1, p2, d1, p3, d2, d3, d4 = bits
    
    s1 = p1 ^ d1 ^ d2 ^ d4
    s2 = p2 ^ d1 ^ d3 ^ d4
    s3 = p3 ^ d2 ^ d3 ^ d4
    syndrome = s1 * 1 + s2 * 2 + s3 * 4
    
    print(f"קוד: {code}")
    print(f"סינדרום: {syndrome}, זוגיות כללית: {'אי-זוגי' if overall_parity else 'זוגי'}")
    
    if syndrome == 0 and overall_parity == 0:
        print("תקין - אין שגיאות")
    elif syndrome != 0 and overall_parity == 1:
        print(f"שגיאה בודדת במיקום {syndrome} - ניתן לתקן")
        bits[syndrome - 1] ^= 1
    elif syndrome == 0 and overall_parity == 1:
        print("שגיאה בביט הזוגיות הכללי")
    else:  # syndrome != 0 and overall_parity == 0
        print("שגיאה כפולה! - לא ניתן לתקן")
        return None
    
    return f"{bits[2]}{bits[4]}{bits[5]}{bits[6]}"

# דוגמאות
print("=== קידוד (8,4) ===")
encoded = hamming_84_encode("1010")
print(f"1010 -> {encoded}\n")

print("=== פענוח תקין ===")
data = hamming_84_decode(encoded)
print(f"מידע: {data}\n")

print("=== שגיאה בודדת ===")
single_error = encoded[:3] + ('0' if encoded[3] == '1' else '1') + encoded[4:]
data = hamming_84_decode(single_error)
print(f"מידע: {data}")

## 7. חישוב מספר ביטי בדיקה

In [ ]:
import math

def min_parity_bits(k):
    """מספר ביטי בדיקה מינימלי עבור k ביטי מידע"""
    r = 1
    while 2**r < k + r + 1:
        r += 1
    return r

# טבלה
print("ביטי מידע | ביטי בדיקה | אורך קוד | יעילות")
print("-" * 50)
for k in [1, 4, 8, 16, 32, 64, 128, 256]:
    r = min_parity_bits(k)
    n = k + r
    efficiency = k / n * 100
    print(f"{k:10} | {r:11} | {n:9} | {efficiency:.1f}%")

## 8. סיכום

| קוד | n | k | d_min | זיהוי | תיקון |
|-----|---|---|-------|-------|-------|
| Parity | k+1 | k | 2 | 1 | 0 |
| Hamming(7,4) | 7 | 4 | 3 | 2 | 1 |
| Hamming(8,4) | 8 | 4 | 4 | 3 | 1* |

\* עם זיהוי שגיאה כפולה (SECDED)